### 최대 크롤링 될 수 있는 지는 모르겠지만...조심해서 사용하셔야합니다..평생 유튜브 못봅니다.

In [1]:
# pip install google-api-python-client
# pip install selenium
# pip install webdriver_manager

In [2]:
import pandas as pd
from googleapiclient.discovery import build

### 댓글 및 스크롤 Counts

In [3]:
from googleapiclient.discovery import build

def count_youtube_comments(api_key, video_id):
    # YouTube API를 사용하여 YouTube 리소스를 생성합니다.
    youtube = build('youtube', 'v3', developerKey=api_key)

    # 댓글 수를 저장할 변수를 초기화합니다.
    comments_count = 0

    # 스크롤 횟수를 저장할 변수를 초기화합니다.
    scroll_count = 0

    # 다음 페이지 토큰을 저장할 변수를 초기화합니다.
    next_page_token = None

    # 무한 루프를 돌면서 댓글을 가져옵니다.
    while True:
        # YouTube API를 사용하여 댓글 스레드 목록을 가져옵니다.
        comments = youtube.commentThreads().list(
            part="snippet",
            videoId=video_id,
            maxResults=100,  # 한 번에 가져올 최대 댓글 수는 100개입니다.
            pageToken=next_page_token
        ).execute()

        # 현재 페이지에서 가져온 댓글 수를 전체 댓글 수에 더합니다.
        comments_count += len(comments['items'])

        # 다음 페이지의 토큰을 가져옵니다.
        next_page_token = comments.get('nextPageToken')

        # 스크롤 횟수를 갱신합니다.
        scroll_count += 1

        # 다음 페이지 토큰이 없으면 더 이상 가져올 댓글이 없으므로 반복을 종료합니다.
        if not next_page_token:
            break

    # 전체 댓글 수와 스크롤 횟수를 반환합니다.
    return comments_count, scroll_count

# 사용 예시                                           '동영상_ID_여기에_입력'이란? https://www.youtube.com/watch?v=htn2CxoqOQY 여기서 v= 뒤의 htn2CxoqOQY를 말합니다.
video_comments_count, scroll_count = count_youtube_comments("API_키_여기에_입력", "동영상_ID_여기에_입력")
print("동영상 댓글 수:", video_comments_count)
print("스크롤 횟수:", scroll_count)


HttpError: <HttpError 400 when requesting https://youtube.googleapis.com/youtube/v3/commentThreads?part=snippet&videoId=%EB%8F%99%EC%98%81%EC%83%81_ID_%EC%97%AC%EA%B8%B0%EC%97%90_%EC%9E%85%EB%A0%A5&maxResults=100&key=API_%ED%82%A4_%EC%97%AC%EA%B8%B0%EC%97%90_%EC%9E%85%EB%A0%A5&alt=json returned "API key not valid. Please pass a valid API key.". Details: "[{'message': 'API key not valid. Please pass a valid API key.', 'domain': 'global', 'reason': 'badRequest'}]">

# Youtube 크롤링

In [ ]:
import csv
from googleapiclient.discovery import build

# max_results_per_request: 각 API 요청에서 가져올 최대 댓글 수(기본 100), max_total_results: 최대로 가져올 총 댓글 수 (기본값 1000)
def crawl_youtube_comments(api_key, video_id, max_results_per_request=100, max_total_results=1000):
    # YouTube API를 사용하여 YouTube 리소스를 생성합니다.
    youtube = build('youtube', 'v3', developerKey=api_key) # Api 관리 들어가시면 보입니다.

    # 동영상의 정보를 가져옵니다.
    video_response = youtube.videos().list(
        part='snippet,statistics',
        id=video_id
    ).execute()

    # 동영상 제목과 조회수를 추출합니다.
    title = video_response['items'][0]['snippet']['title']
    view_count = video_response['items'][0]['statistics']['viewCount']

    # CSV 파일을 쓰기 모드로 엽니다.(w = 쓰기, r = 읽기, a = 추가)
    with open('comments.csv', 'w', newline='', encoding='utf-8') as csvfile:
        # CSV 파일의 헤더를 설정합니다.
        fieldnames = ['video_title', 'video_view_count', 'author', 'comment_date', 'comment', 'reply_author', 'reply_date', 'reply_comment']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 헤더를 씁니다.(컬럼)
        writer.writeheader()

        # next_page_token = None: YouTube API의 페이지네이션을 위해 사용되는 토큰
        # 댓글을 가져오기 위한 변수들을 초기화합니다.
        total_results = 0
        next_page_token = None  # 다음 페이지 토큰 초기화

        # 최대 댓글 수를 넘지 않을 때까지 반복합니다.
        while total_results < max_total_results:
            # YouTube API를 사용하여 댓글 스레드 목록을 가져옵니다.
            comments = youtube.commentThreads().list(
                part="snippet",
                videoId=video_id,
                maxResults=max_results_per_request,
                order="relevance",
                pageToken=next_page_token  # 다음 페이지 토큰 설정
            ).execute()

            # 가져온 각 댓글에 대해 처리합니다.
            for comment in comments['items']:
                # 댓글 정보를 추출합니다.
                comment_text = comment['snippet']['topLevelComment']['snippet']['textDisplay']
                author = comment['snippet']['topLevelComment']['snippet']['authorDisplayName']
                comment_date = comment['snippet']['topLevelComment']['snippet']['publishedAt']
                reply_count = comment['snippet']['totalReplyCount']

                # 댓글을 CSV 파일에 작성합니다.
                writer.writerow({
                    'video_title': title,
                    'video_view_count': view_count,
                    'author': author,
                    'comment_date': comment_date,
                    'comment': comment_text
                })

                # 대댓글이 있는 경우 반복문 실행
                if reply_count > 0:
                    # 대댓글을 가져옵니다.
                    replies = youtube.comments().list(
                        part="snippet",
                        parentId=comment['id'],
                        maxResults=reply_count
                    ).execute()
                    # 각 대댓글에 대해 처리합니다.

                    for reply in replies['items']:
                        reply_text = reply['snippet']['textDisplay']
                        reply_author = reply['snippet']['authorDisplayName']
                        reply_date = reply['snippet']['publishedAt']

                        # 대댓글을 CSV 파일에 작성합니다.
                        # 여기같은 경우는 대댓글을 따로 열을 뺏습니다. 그래서 댓글 - 답글 순인데 댓글에서 Nan이 나옵니다.
                        writer.writerow({
                            'video_title': '',
                            'video_view_count': '',
                            'author': reply_author,
                            'comment_date': reply_date,
                            'comment': '',
                            'reply_author': reply_author,
                            'reply_date': reply_date,
                            'reply_comment': reply_text
                        })

            # 가져온 댓글 수를 업데이트하고 다음 페이지 토큰을 설정합니다.
            total_results += len(comments['items'])    # 가져온 댓글의 수를 누적하여 전체 댓글의 수를 계산, 중복된 댓글은 무시되고 각 댓글은 한 번씩만 누적
            next_page_token = comments.get('nextPageToken')

            # 최대 댓글 수에 도달하거나 다음 페이지 토큰이 없으면 반복을 종료합니다.
            if total_results >= max_total_results or not next_page_token:   # 현재까지 가져온 댓글의 총 개수가 최대 허용되는 댓글 수(max_total_results)를 초과했는지를 확인
                break # 절대 절대 절대로 멈춰야 합니다.

    # 동영상 제목과 조회수를 반환합니다.
    return title, view_count

# 함수를 사용하여 동영상 정보를 가져옵니다.              '동영상_ID_여기에_입력'이란? https://www.youtube.com/watch?v=htn2CxoqOQY 여기서 v= 뒤의 htn2CxoqOQY를 말합니다.
video_title, video_view_count = crawl_youtube_comments("API_키_여기에_입력", "동영상_ID_여기에_입력", max_total_results=1400)

# 결과를 출력합니다.
print("동영상 제목:", video_title)
print("동영상 조회수:", video_view_count)

In [ ]:
df = pd.read_csv('KBS_라인_뺏기나.csv')

In [ ]:
df.info()

In [ ]:
df.set_index(['video_title', 'video_view_count'], inplace=True)

In [ ]:
df.head()